### Analise Operacional
Essa analise busca explorar as deficiências e eficiências de um Dataset fictício voltado ao operacional de uma empresa, através de uma sequência de chamados de suporte. 

Os dados coletados na tabela de exemplo são:

| Nome da coluna          | Descrição                                                   |
| ----------------------- | ----------------------------------------------------------- |
| Ticket                  | Identificador único do ticket                               |
| Sobrenome               | Sobrenome do solicitante                                    |
| Departamento            | Departamento relacionado ao ticket                          |
| Grupo                   | Grupo responsável ou segmentação interna                    |
| Estado Civil            | Estado civil do solicitante                                 |
| Idade                   | Idade do solicitante                                        |
| Unidade                 | Localidade ou unidade organizacional                        |
| Gênero                  | Gênero do solicitante                                       |
| Classificação de Cargos | Cargo, função ou classificação do colaborador               |
| Senioridade             | Nível hierárquico/senioridade do profissional               |
| Responsável de TI       | Profissional de TI responsável pelo ticket                  |
| Arquivado_Como          | Categorização do ticket                                     |
| TicketTipo              | Tipo de ticket (ex: incidente, solicitação, requisição)     |
| Severidade              | Grau de severidade do ticket                                |
| Prioridade              | Prioridade atribuída ao ticket                              |
| DiasAberto              | Quantidade de dias que o ticket ficou aberto                |
| Satisfação              | Grau de satisfação registrado pelo usuário após atendimento |

### **Tempo Médio de Atendimento (TMA)**

Descreve a média de tempo que os clientes levam para ter seus tikets concluídos. 

***Interpretação***

O tempo médio de 6,84 dias indica que o ciclo de resolução não é crítico, mas está acima do ideal para operações de Help Desk que buscam agilidade. O número sugere que não há gargalo grave, porém a ausência de padronização entre equipes pode estar elevando a média.


In [0]:
SELECT ROUND(AVG(dias_aberto),2) AS media_dias_aberto
FROM workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk;

media_dias_aberto
6.84


### **Desvio Padrão/Variância no Atendimento**

Descreve o desvio padrão e Variância sobre os dias abertos por chamado (ticket)

***Interpretação:***

O desvio padrão de 7,38 dias, maior do que a própria média, indica alta dispersão.
Isso significa que alguns tickets estão sendo resolvidos rápido (baixo SLA), enquanto outros ficam muito acima do aceitável. Esse comportamento evidencia **ausência de padronização**, possível sobrecarga pontual ou falha no direcionamento dos casos mais complexos.


In [0]:
WITH desvios_aberto AS (
SELECT
  ticket,
  dias_aberto,
  ROUND(AVG(dias_aberto) OVER (), 2) AS media_aberta, -- média de dias abertos
  ROUND(dias_aberto - ROUND(AVG(dias_aberto) OVER (), 2),2) AS desvio, -- desvios em relação a média
  ROUND(ROUND(dias_aberto - ROUND(AVG(dias_aberto) OVER (), 2),2) * ROUND(dias_aberto - ROUND(AVG(dias_aberto) OVER (), 2),2),2) AS desvio_quadrado -- desvios ao quadrado
FROM workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk
)
SELECT 
  ROUND(SUM(desvio_quadrado) / COUNT(*),2) AS variancia,
  ROUND(SQRT(SUM(desvio_quadrado) / COUNT(*)),2) AS desvio_padrao
FROM desvios_aberto;

col
1
3
5
7
9


### **Ranking Departamentos**

Tabela com os departamentos com maior quantidade de Tickets Abertos

***Interpretação:***

Departamentos com maior volume (Finanças, Suporte Técnico, RH) apresentam melhor média de resolução, o que indica maior eficiência operacional em escala (um sinal positivo).
Entretanto, os departamentos com poucos tickets têm tempos médios piores, o que é um indicador típico de que o problema pode não estar apenas no volume, mas na maturidade do fluxo de atendimento por área.

In [0]:
SELECT ROW_NUMBER() OVER (ORDER BY COUNT(departamento) DESC) AS ranking, 
  departamento,
  COUNT (departamento) AS contagem,
  ROUND(AVG (dias_aberto),2) AS media_dias_aberto
FROM workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk
GROUP BY departamento
ORDER By contagem DESC;

ranking,departamento,contagem,media_dias_aberto
1,Finanças,7302,6.83
2,Suporte técnico,7212,6.68
3,Recursos Humanos,7186,6.85
4,Anúncio,7036,6.91
5,Pesquisa e Desenvolvimento,6973,6.67
6,Relações com a mídia,6892,6.83
7,Folha de pagamento,6890,6.9
8,Departamento Jurídico,6791,6.85
9,Garantia de Qualidade,6616,7.05
10,Vendas e Marketing,6538,6.75


### **Desempenho por Severidade**

Média de dias abertos de acordo com a sua severidade + média de satisfação do chamado

***Interpretação:***

Chamados de severidade baixa têm SLA melhor, porém satisfação pior. Isso indica que o tempo não é o único determinante da experiência do usuário.

Possíveis causas:
- má comunicação;
- baixa clareza da solução entregue;
- tickets recorrentes / irritantes;
- resolução rápida, mas superficial.

Já severidades mais altas têm SLA pior porém satisfação maior, sugerindo que os analistas dedicam mais cuidado e comunicação nos casos críticos.
Isso reforça a necessidade de revisar o modelo de priorização e o treinamento operacional.

**Preliminar: Criação da Coluna `nivel_satisfacao`**

1. Adicionar a coluna
```sql
ALTER TABLE workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk
ADD COLUMNS (nivel_satisfacao INT);
```

2. Atualizar valores com CASE WHEN
```sql
UPDATE workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk
SET nivel_satisfacao = 
    CASE 
        WHEN satisfacao = '0 - Desconhecido' THEN 0
        WHEN satisfacao = '1 - Insatisfeito' THEN 1
        WHEN satisfacao = '2 - Satisfeito' THEN 2
        WHEN satisfacao = '3 - Muito Satisfeito' THEN 3 
    END;
```

In [0]:
SELECT severidade, 
  ROUND (AVG(nivel_satisfacao),2) AS media_satisfacao,
  ROUND (AVG(dias_aberto),2) AS media_dias_aberto
FROM workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk
GROUP BY severidade
ORDER BY severidade ASC;

severidade,media_satisfacao,media_dias_aberto
0 - Unclassified,1.4,4.07
1 - Menor,1.33,5.08
2 - Normal,1.47,6.88
3 - Maior,1.61,7.2
4 - Crítico,1.6,6.51


### **Percentis de Dias Abertos — 25%, 50%, 75% e 90% (Por Severidade)**

Levantamento dos percentis 25%, 50%, 75% e 90% para melhor entendimento da distribuição dos dias abertos dos tickets em questão

***Interpretação:***

Para severidades mais críticas, o percentil 90 é significativamente maior do que nas severidades menores, mostrando uma cauda longa de tickets que demoram demais.
Isso afeta diretamente SLA e percepção de eficiência.

O percentil 50 (mediana) também sendo mais alto nas severidades críticas sugere que o problema não ocorre apenas em casos excepcionais: há lentidão estrutural em demandas urgentes.

In [0]:
 SELECT COALESCE (severidade, 'Geral') AS severidade, 
  PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY dias_aberto) AS percentil_25, 
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dias_aberto) AS mediana,
  PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY dias_aberto) AS percentil_75,
  PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY dias_aberto) AS percentil_90
FROM workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk
GROUP BY ROLLUP (severidade)
ORDER BY severidade ASC;

severidade,percentil_25,mediana,percentil_75,percentil_90
0 - Unclassified,0.0,2.0,6.0,10.0
1 - Menor,0.0,3.0,7.0,13.0
2 - Normal,1.0,5.0,10.0,17.0
3 - Maior,1.0,5.0,12.0,18.0
4 - Crítico,1.0,4.0,10.0,17.0
Geral,1.0,5.0,10.0,17.0


### **Dias Abertos por Severidade**

Gráfico box com os dias abertos, de acordo com as principais severidades

***Interpretação:***

Representação gráfica da analise anterior: Note como os boxes de `0 - Unclassified` e `1 - Menor` estão localizados em uma posição mais abaixo que os demais.

In [0]:
SELECT 
      ticket,
      severidade,
      dias_aberto
      FROM workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk;

ticket,severidade,dias_aberto
13461,2 - Normal,1
52458,2 - Normal,0
34480,2 - Normal,0
42415,2 - Normal,10
68802,2 - Normal,29
83618,2 - Normal,0
23607,3 - Maior,10
75855,2 - Normal,1
83790,2 - Normal,18
91725,2 - Normal,11


Databricks visualization. Run in Databricks to view.

### **Chamados por Assunto**

Quantidade de chamados de acordo com cada assunto

***Interpretação:***

"Sistema" concentra o maior volume de tickets, o que pode indicar:
- instabilidade recorrente;
- dificuldade de usabilidade;
- falta de automações;
- necessidade de treinamentos.

Já "Hardware" tem poucos tickets, o que indica ambiente físico estável, porém é necessário investigar se o número baixo não vem de subnotificação.

In [0]:
SELECT arquivado_como,
  COUNT(*) AS contagem 
FROM workspace.google_drive.cenario_operacional_help_desk_tickets_it_help_desk
GROUP BY arquivado_como;

arquivado_como,contagem
Acesso/Login,29921
Hardware,9976
Sistema,40035
Software,20068



## Recomendações Estratégicas:

Com base na alta **dispersão** no atendimento (Desvio Padrão > Média) e na **insatisfação** em serviços rápidos (Baixa Severidade), a estratégia deve focar em padronização e otimização do autoatendimento.

### 1. Pilar: Padronização e Previsibilidade Operacional

Foco em reduzir a alta variabilidade (desvio padrão de 7,38 dias) e a lentidão estrutural em casos urgentes.

| Ação Estratégica | Meta (KPI Tático) | Próximo Passo Imediato |
| :--- | :--- | :--- |
| **Implementação de Playbooks e SLAs Rigorosos** | Reduzir o Desvio Padrão Geral em **25%** em 6 meses. | Análise de **Desvio Padrão por Responsável/Grupo de TI** para isolar as maiores fontes de variação. |
| **Transferência de Melhores Práticas** | Aumentar o TMA das áreas com menor volume em **15%**. | Modelar e replicar os fluxos eficientes dos departamentos de **alto volume** (ex: Finanças, Suporte Técnico). |

### 2. Pilar: Otimização da Experiência via Automação

Abordar a baixa satisfação em tickets de **Baixa Severidade**, liberando a equipe para casos complexos.

| Ação Estratégica | Meta (KPI Tático) | Próximo Passo Imediato |
| :--- | :--- | :--- |
| **Adoção de Self-Service e Chatbots** | Reduzir o volume de tickets de Baixa Severidade em **40%** em 3 meses. | Investimento na criação de uma **Base de Conhecimento (FAQ)** e canais de autoatendimento para problemas recorrentes em "Sistema". |
| **Melhoria da Qualidade de Atendimento** | Aumentar a Satisfação Média (Nível 1 e 2) em **0.5 ponto**. | Treinamento da equipe para focar na **clareza da solução e comunicação proativa**, indo além da métrica de tempo. |

### 3. Pilar: Mitigação de Risco em Incidentes Críticos

Eliminar a "cauda longa" de tickets de alta severidade (percentil P90), que ameaçam a continuidade do negócio.

| Ação Estratégica | Meta (KPI Tático) | Próximo Passo Imediato |
| :--- | :--- | :--- |
| **Revisão do Fluxo de Escalamento Crítico** | Garantir que o Percentil 90 dos tickets críticos não ultrapasse o **P75** atual. | Mapeamento e otimização dos pontos de escalonamento para garantir que a **ação de emergência** seja disparada mais rapidamente. |